# Keras によるモデル開発 (リモート環境)

Keras を利用したモデル学習を行います。犬、猫の画像を区別する画像認識モデルを構築します。Datasetは各ラベルごとに学習用・検証用に分けて登録します。

## Azure Machine Learning Python SDK インポート

In [1]:
from azureml.core import Workspace, Dataset, Experiment
print(azureml.core.VERSION)

1.0.65


## Azure Machine Learning Workspace への接続

In [2]:
subscription_id = '9c0f91b8-eb2f-484c-979c-15848c098a6b'
resource_group = 'dllab-test'
workspace_name = 'azureml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

## 計算環境 Machine Learning Compute のアタッチ

In [3]:
from azureml.core.compute import ComputeTarget
compute_target = ComputeTarget(workspace,"gpu-clst")

## 実験名の設定

In [4]:
experiment = Experiment(workspace = workspace, name = "keras_catdog_remote")

## 学習データ
事前に Azure Machine Learning service Workspace に Dataset を登録しておきます。

In [5]:
cat_train = Dataset.get_by_name(workspace, name='cat_train')
dog_train = Dataset.get_by_name(workspace, name='dog_train')
cat_test = Dataset.get_by_name(workspace, name='cat_test')
dog_test = Dataset.get_by_name(workspace, name='dog_test')

## モデル学習コード

In [9]:
import os
project_folder="./keras_remote2"
os.makedirs(project_folder, exist_ok=True)

In [10]:
%%writefile {project_folder}/keras_dogcat.py

import numpy as np
import tensorflow as tf

from PIL import Image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import array_to_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Input, Dense
from tensorflow.python.keras.models import Model
import os
from azureml.core import Run
from azureml.core import Workspace, Dataset
from keras.utils import plot_model
import argparse

print("## START Script ##")


parser = argparse.ArgumentParser()
parser.add_argument('--batch-size', type=int, dest='batch_size', default=16, help='mini batch size for training')
parser.add_argument('--learning-rate', type=float, dest='learning_rate', default=0.01, help='learning rate')
parser.add_argument('--dropout', type=float, dest='dropout', default=0.01, help='dropout rate')
parser.add_argument('--epoch', type=int, dest='epoch', default=10, help='number of epoch')


args = parser.parse_args()



batch_size = args.batch_size
learing_rate = args.learning_rate
dropout = args.dropout
epoch = args.epoch

run = Run.get_context()

os.makedirs('train', exist_ok=True)
cat_train = run.input_datasets['cat_train'].download(target_path='train/cat/', overwrite=False)
dog_train = run.input_datasets['dog_train'].download(target_path='train/dog/', overwrite=False)

os.makedirs('test', exist_ok=True)
cat_test = run.input_datasets['cat_test'].download(target_path='test/cat/', overwrite=False)
dog_test = run.input_datasets['dog_test'].download(target_path='test/dog/', overwrite=False)

print(os.listdir())
print(os.listdir("train"))
print(os.listdir("test"))
print(os.listdir("train/cat/"))
print(os.listdir("train/dog/"))
print(os.listdir("test/cat/"))
print(os.listdir("test/dog/"))


from pip._internal.operations.freeze import freeze

class RunCallback(tf.keras.callbacks.Callback):
    def __init__(self, run):
        self.run = run
        
    def on_epoch_end(self, batch, logs={}):
        self.run.log(name="training_acc", value=float(logs.get('acc')))
        self.run.log(name="validation_acc", value=float(logs.get('val_acc')))
    

inputs = Input(shape=(150, 150, 3))
x = Conv2D(32, (3, 3))(inputs)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(32, (3, 3))(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3))(x)
x = Activation("relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(dropout)(x)
prediction = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=prediction)
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])


callbacks = list()
callbacks.append(RunCallback(run))

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    './train',
    target_size=(150, 150), # resize
    batch_size=batch_size,
    class_mode="binary")

validation_generator = test_datagen.flow_from_directory(
     './test',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode="binary")

VERBOSE = 1

print("## START TRAINING ##")

model.fit_generator(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=epoch,
    validation_data=validation_generator,
    validation_steps=800 // batch_size,
    callbacks= callbacks)

run.log("Batch size",batch_size)
run.log("Num of Epoch", epoch)
run.log("Dropout Rate", dropout)

model.save_weights("./outputs/keras_simple.h5")
#plot_model(model, to_file='./outputs/model.png')


Writing ./keras_remote2/keras_dogcat.py


## Estimator 設定

In [11]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--batch-size': 32,
    '--learning-rate': 0.0001,
    '--dropout': 0.03,
    '--epoch':10
}


estimator = TensorFlow(source_directory=project_folder,
                       script_params = script_params,
                       compute_target=compute_target,
                       entry_script='keras_dogcat.py',
                       framework_version = '1.13',
                       pip_packages=['keras','Pillow','azureml-dataprep[pandas,fuse]'],
                       inputs=[cat_train.as_named_input('cat_train'),
                               dog_train.as_named_input('dog_train'),
                               cat_test.as_named_input('cat_test'),
                               dog_test.as_named_input('dog_test'),]
                      )

## モデル学習と結果確認

In [12]:
run = experiment.submit(estimator)

In [18]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## モデル登録

In [19]:
model = run.register_model(model_name='tf-catdog-remote', model_path='outputs/keras_simple.h5')
print(model.name, model.id, model.version, sep = '\t')

tf-catdog-remote	tf-catdog-remote:6	6


In [20]:
run.complete()

In [21]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
keras_catdog_remote,keras_catdog_remote_1570499528_c1085348,azureml.scriptrun,Finalizing,Link to Azure Portal,Link to Documentation
